In [1]:
# 필요한 라이브러리를 불러들임 
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# MNIST 데이터 불러오기
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# 그리드 서치를 통한 두 번째와 세 번째 은닉층의 노드 수 탐색
for n_hidden2 in range(128, 449, 64): # 128~448까지 64씩 증가
    for n_hidden3 in range(128, 449, 64):
        # 학습 관련 매개변수 및 모델 구조 정의
        n_input = 784
        n_hidden1 = 400
        n_hidden4 = 50
        display_step = 1
        n_epoch = 200
        batch_size = 256
        lr_rbm = tf.constant(0.001, tf.float32)
        lr_class = tf.constant(0.01, tf.float32)
        n_class = 10
        n_iter = 200

        # 입력 및 출력 정의
        x = tf.placeholder(tf.float32, [None, n_input], name="x")
        y = tf.placeholder(tf.float32, [None, 10], name="y")

        # 첫 번째 은닉층 가중치 및 편향 정의
        W1 = tf.Variable(tf.random_normal([n_input, n_hidden1], 0.01), name="W1")
        b1_h = tf.Variable(tf.zeros([1, n_hidden1], tf.float32, name="b1_h"))
        b1_i = tf.Variable(tf.zeros([1, n_input], tf.float32, name="b1_i"))

        # 두 번째 은닉층 가중치 및 편향 정의
        W2 = tf.Variable(tf.random_normal([n_hidden1, n_hidden2], 0.01), name="W2")
        b2_h = tf.Variable(tf.zeros([1, n_hidden2], tf.float32, name="b2_h"))
        b2_i = tf.Variable(tf.zeros([1, n_hidden1], tf.float32, name="b2_i"))

        # 세 번째 은닉층 가중치 및 편향 정의
        W3 = tf.Variable(tf.random_normal([n_hidden2, n_hidden3], 0.01), name="W3")
        b3_h = tf.Variable(tf.zeros([1, n_hidden3], tf.float32, name="b3_h"))
        b3_i = tf.Variable(tf.zeros([1, n_hidden2], tf.float32, name="b3_i"))

        # 네 번째 은닉층 가중치 및 편향 정의
        W4 = tf.Variable(tf.random_normal([n_hidden3, n_hidden4], 0.01), name="W4")
        b4_h = tf.Variable(tf.zeros([1, n_hidden4], tf.float32, name="b4_h"))
        b4_i = tf.Variable(tf.zeros([1, n_hidden3], tf.float32, name="b4_i"))

        # 라벨층 가중치 및 편향 정의
        W_c = tf.Variable(tf.random_normal([n_hidden4, n_class], 0.01), name="W_c")
        b_c = tf.Variable(tf.zeros([1, n_class], tf.float32, name="b_c"))

        # 확률을 이산 상태, 즉 0과 1로 변환함 
        def binary(prob):
            return tf.floor(prob + tf.random_uniform(tf.shape(prob), 0, 1))
        
        # Gibbs 표본추출 단계
        def cd_step(x_k,W,b_h,b_i):
            h_k = binary(tf.sigmoid(tf.matmul(x_k, W) + b_h)) 
            x_k = binary(tf.sigmoid(tf.matmul(h_k, tf.transpose(W)) + b_i))
            return x_k
        
        # 표본추출 단계 실행     
        def cd_gibbs(k,x_k,W,b_h,b_i):
            for i in range(k):
                x_out = cd_step(x_k,W,b_h,b_i) 
            # k 반복 후에 깁스 표본을 반환함
            return x_out
        
        # 4개의 은닉층을 갖는 DBN에 대한 CD-2 알고리즘
        # 1. 현재 입력값을 기반으로 깁스 표본추출을 통해 새로운 입력값 x_s를 구함
        # 2. 새로운 x_s를 기반으로 새로운 은닉노드 값 h_s를 구함    
        x_s = cd_gibbs(2,x,W1,b1_h,b1_i) 
        act_h1_s = binary(tf.sigmoid(tf.matmul(x_s, W1) + b1_h)) 
        h1_s = cd_gibbs(2,act_h1_s,W2,b2_h,b2_i) 
        act_h2_s = binary(tf.sigmoid(tf.matmul(h1_s, W2) + b2_h)) 
        h2_s = cd_gibbs(2, act_h2_s, W3, b3_h, b3_i)  
        act_h3_s = binary(tf.sigmoid(tf.matmul(h2_s, W3) + b3_h))
        h3_s = cd_gibbs(2, act_h3_s, W4, b4_h, b4_i)  
        act_h4_s = binary(tf.sigmoid(tf.matmul(h3_s, W4) + b4_h))
        
        # 입력값이 주어질 때 은닉노드 값 h를 구함
        act_h1 = tf.sigmoid(tf.matmul(x, W1) + b1_h) 
        act_h2 = tf.sigmoid(tf.matmul(act_h1_s, W2) + b2_h)
        act_h3 = tf.sigmoid(tf.matmul(act_h2_s, W3) + b3_h) 
        act_h4 = tf.sigmoid(tf.matmul(act_h3_s, W4) + b4_h) 
        
        # 경사 하강법을 이용한 가중치 및 편향 업데이트 
        size_batch = tf.cast(tf.shape(x)[0], tf.float32)
        # 첫 번째 은닉층
        W1_add  = tf.multiply(lr_rbm/size_batch, tf.subtract(tf.matmul(tf.transpose(x), \
                  act_h1), tf.matmul(tf.transpose(x_s), act_h1_s)))
        b1_i_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(x, x_s), \
                   0, True))
        b1_h_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(act_h1, act_h1_s), \
                   0, True))
        
        # 두 번째 은닉층
        W2_add  = tf.multiply(lr_rbm/size_batch, tf.subtract(tf.matmul(tf.transpose(act_h1_s), \
                  act_h2), tf.matmul(tf.transpose(h1_s), act_h2_s)))
        b2_i_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(act_h1_s, h1_s), \
                   0, True))
        b2_h_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(act_h2, act_h2_s), \
                0, True))
        
        # 세 번째 은닉층
        W3_add  = tf.multiply(lr_rbm/size_batch, tf.subtract(tf.matmul(tf.transpose(act_h2_s), \
                  act_h3), tf.matmul(tf.transpose(h2_s), act_h3_s)))  
        b3_i_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(act_h2_s, h2_s), \
                   0, True))  
        b3_h_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(act_h3, act_h3_s), \
                 0, True))  
        
        # 네 번째 은닉층
        W4_add  = tf.multiply(lr_rbm/size_batch, tf.subtract(tf.matmul(tf.transpose(act_h3_s), \
                  act_h4), tf.matmul(tf.transpose(h3_s), act_h4_s)))  
        b4_i_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(act_h3_s, h3_s), \
                   0, True))  
        b4_h_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(act_h4, act_h4_s), \
                 0, True))  
        
        updt = [W1.assign_add(W1_add), b1_i.assign_add(b1_i_add), b1_h.assign_add(b1_h_add),\
                W2.assign_add(W2_add), b2_i.assign_add(b2_i_add), b2_h.assign_add(b2_h_add),\
                W3.assign_add(W3_add), b3_i.assign_add(b3_i_add), b3_h.assign_add(b3_h_add),\
                W4.assign_add(W4_add), b4_i.assign_add(b4_i_add), b4_h.assign_add(b4_h_add)] 
        
        #-------------------------------------------------------------
        # 소프트맥스 층을 추가한 분류용-DBN 을 위한 연산과정
        #------------------------------------------------------------- 
        
        logits = tf.matmul(act_h4,W_c) + b_c
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))
        optimizer = tf.train.AdamOptimizer(learning_rate=lr_class).minimize(cost)
        correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        #-------------------------------------------------------------                   
        # RBM을 쌓아 올려가며 DBN을 학습하는 텐서플로 그래프 실행   
        #-------------------------------------------------------------
        with tf.Session() as sess:
            # Initialize the variables of the Model
            init = tf.global_variables_initializer()
            sess.run(init)
            
            n_batch = int(len(x_train)/batch_size)
            # Start the training 
            for epoch in range(n_epoch):
                # Loop over all batches
                for i in range(n_batch):
                    batch_xs = x_train[i*batch_size:(i+1)*batch_size]
                    # ++ 이미지 데이터를 784차원 벡터로 평탄화
                    batch_xs = batch_xs.reshape((-1, 784))
                    batch_xs = (batch_xs > 0)*1
                    # Run the weight update 
                    _ = sess.run(updt, feed_dict={x:batch_xs})
                    
                # Display the running step 
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1))
                          
            print("RBM training Completed !")
        #--------------------------------------------------------------
        # 소프트맥스 층을 추가한 분류용-DBN 학습 및 예측
        #--------------------------------------------------------------
            for i in range(n_iter):
                indices = np.random.randint(len(x_train), size=batch_size)
                batch_x = x_train[indices]
                batch_y = y_train[indices]
        
                # ++ 이미지 데이터를 784차원 벡터로 평탄화
                batch_x = batch_x.reshape((-1, 784))
                # ++ 레이블 데이터를 원-핫 인코딩
                batch_y = np.eye(10)[batch_y]
            
                # 최적화 과정 실행
                sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
                if i % 10 == 0:
                    # MINIST 훈련용 이미지의 배치에 대한 손실과 정확도를 계산
                    tr_loss, tr_acc = sess.run([cost, accuracy], 
                    	                    feed_dict={x: batch_x, y: batch_y})
                    print("Iter " + str(i) + ", Minibatch Loss= " + \
                          "{:.6f}".format(tr_loss) + ", Training Accuracy= " + \
                          "{:.5f}".format(tr_acc))
                
            print("Optimization Finished!")
        
            # MINIST 검정용 이미지에 대한 정확도 계산 
            print("Testing Accuracy:", \
                sess.run(accuracy, feed_dict={x: x_test.reshape((-1, 784)), #타입 수정
                                              y: np.eye(10)[y_test]})) #타입 수정
        
            sess.close()
             
        print("Hidden Layer 2 Nodes:", n_hidden2, "Hidden Layer 3 Nodes:", n_hidden3, "Accuracy:", accuracy)


Epoch: 0001
Epoch: 0002
Epoch: 0003
Epoch: 0004
Epoch: 0005
Epoch: 0006
Epoch: 0007
Epoch: 0008
Epoch: 0009
Epoch: 0010
Epoch: 0011
Epoch: 0012
Epoch: 0013
Epoch: 0014
Epoch: 0015
Epoch: 0016
Epoch: 0017
Epoch: 0018
Epoch: 0019
Epoch: 0020
Epoch: 0021
Epoch: 0022
Epoch: 0023
Epoch: 0024
Epoch: 0025
Epoch: 0026
Epoch: 0027
Epoch: 0028
Epoch: 0029
Epoch: 0030
Epoch: 0031
Epoch: 0032
Epoch: 0033
Epoch: 0034
Epoch: 0035
Epoch: 0036
Epoch: 0037
Epoch: 0038
Epoch: 0039
Epoch: 0040
Epoch: 0041
Epoch: 0042
Epoch: 0043
Epoch: 0044
Epoch: 0045
Epoch: 0046
Epoch: 0047
Epoch: 0048
Epoch: 0049
Epoch: 0050
Epoch: 0051
Epoch: 0052
Epoch: 0053
Epoch: 0054
Epoch: 0055
Epoch: 0056
Epoch: 0057
Epoch: 0058
Epoch: 0059
Epoch: 0060
Epoch: 0061
Epoch: 0062
Epoch: 0063
Epoch: 0064
Epoch: 0065
Epoch: 0066
Epoch: 0067
Epoch: 0068
Epoch: 0069
Epoch: 0070
Epoch: 0071
Epoch: 0072
Epoch: 0073
Epoch: 0074
Epoch: 0075
Epoch: 0076
Epoch: 0077
Epoch: 0078
Epoch: 0079
Epoch: 0080
Epoch: 0081
Epoch: 0082
Epoch: 0083
Epo

In [3]:
for n_hidden3 in range(320, 449, 64): # n_hidden2 = 448일 때, 320~448까지 64씩 증가
        # 학습 관련 매개변수 및 모델 구조 정의
        n_input = 784
        n_hidden1 = 400
        n_hidden2 = 448
        n_hidden4 = 50
        display_step = 1
        n_epoch = 200
        batch_size = 256
        lr_rbm = tf.constant(0.001, tf.float32)
        lr_class = tf.constant(0.01, tf.float32)
        n_class = 10
        n_iter = 200

        # 입력 및 출력 정의
        x = tf.placeholder(tf.float32, [None, n_input], name="x")
        y = tf.placeholder(tf.float32, [None, 10], name="y")

        # 첫 번째 은닉층 가중치 및 편향 정의
        W1 = tf.Variable(tf.random_normal([n_input, n_hidden1], 0.01), name="W1")
        b1_h = tf.Variable(tf.zeros([1, n_hidden1], tf.float32, name="b1_h"))
        b1_i = tf.Variable(tf.zeros([1, n_input], tf.float32, name="b1_i"))

        # 두 번째 은닉층 가중치 및 편향 정의
        W2 = tf.Variable(tf.random_normal([n_hidden1, n_hidden2], 0.01), name="W2")
        b2_h = tf.Variable(tf.zeros([1, n_hidden2], tf.float32, name="b2_h"))
        b2_i = tf.Variable(tf.zeros([1, n_hidden1], tf.float32, name="b2_i"))

        # 세 번째 은닉층 가중치 및 편향 정의
        W3 = tf.Variable(tf.random_normal([n_hidden2, n_hidden3], 0.01), name="W3")
        b3_h = tf.Variable(tf.zeros([1, n_hidden3], tf.float32, name="b3_h"))
        b3_i = tf.Variable(tf.zeros([1, n_hidden2], tf.float32, name="b3_i"))

        # 네 번째 은닉층 가중치 및 편향 정의
        W4 = tf.Variable(tf.random_normal([n_hidden3, n_hidden4], 0.01), name="W4")
        b4_h = tf.Variable(tf.zeros([1, n_hidden4], tf.float32, name="b4_h"))
        b4_i = tf.Variable(tf.zeros([1, n_hidden3], tf.float32, name="b4_i"))

        # 라벨층 가중치 및 편향 정의
        W_c = tf.Variable(tf.random_normal([n_hidden4, n_class], 0.01), name="W_c")
        b_c = tf.Variable(tf.zeros([1, n_class], tf.float32, name="b_c"))

        # 확률을 이산 상태, 즉 0과 1로 변환함 
        def binary(prob):
            return tf.floor(prob + tf.random_uniform(tf.shape(prob), 0, 1))
        
        # Gibbs 표본추출 단계
        def cd_step(x_k,W,b_h,b_i):
            h_k = binary(tf.sigmoid(tf.matmul(x_k, W) + b_h)) 
            x_k = binary(tf.sigmoid(tf.matmul(h_k, tf.transpose(W)) + b_i))
            return x_k
        
        # 표본추출 단계 실행     
        def cd_gibbs(k,x_k,W,b_h,b_i):
            for i in range(k):
                x_out = cd_step(x_k,W,b_h,b_i) 
            # k 반복 후에 깁스 표본을 반환함
            return x_out
        
        # 4개의 은닉층을 갖는 DBN에 대한 CD-2 알고리즘
        # 1. 현재 입력값을 기반으로 깁스 표본추출을 통해 새로운 입력값 x_s를 구함
        # 2. 새로운 x_s를 기반으로 새로운 은닉노드 값 h_s를 구함    
        x_s = cd_gibbs(2,x,W1,b1_h,b1_i) 
        act_h1_s = binary(tf.sigmoid(tf.matmul(x_s, W1) + b1_h)) 
        h1_s = cd_gibbs(2,act_h1_s,W2,b2_h,b2_i) 
        act_h2_s = binary(tf.sigmoid(tf.matmul(h1_s, W2) + b2_h)) 
        h2_s = cd_gibbs(2, act_h2_s, W3, b3_h, b3_i)  
        act_h3_s = binary(tf.sigmoid(tf.matmul(h2_s, W3) + b3_h))
        h3_s = cd_gibbs(2, act_h3_s, W4, b4_h, b4_i)  
        act_h4_s = binary(tf.sigmoid(tf.matmul(h3_s, W4) + b4_h))
        
        # 입력값이 주어질 때 은닉노드 값 h를 구함
        act_h1 = tf.sigmoid(tf.matmul(x, W1) + b1_h) 
        act_h2 = tf.sigmoid(tf.matmul(act_h1_s, W2) + b2_h)
        act_h3 = tf.sigmoid(tf.matmul(act_h2_s, W3) + b3_h) 
        act_h4 = tf.sigmoid(tf.matmul(act_h3_s, W4) + b4_h) 
        
        # 경사 하강법을 이용한 가중치 및 편향 업데이트 
        size_batch = tf.cast(tf.shape(x)[0], tf.float32)
        # 첫 번째 은닉층
        W1_add  = tf.multiply(lr_rbm/size_batch, tf.subtract(tf.matmul(tf.transpose(x), \
                  act_h1), tf.matmul(tf.transpose(x_s), act_h1_s)))
        b1_i_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(x, x_s), \
                   0, True))
        b1_h_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(act_h1, act_h1_s), \
                   0, True))
        
        # 두 번째 은닉층
        W2_add  = tf.multiply(lr_rbm/size_batch, tf.subtract(tf.matmul(tf.transpose(act_h1_s), \
                  act_h2), tf.matmul(tf.transpose(h1_s), act_h2_s)))
        b2_i_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(act_h1_s, h1_s), \
                   0, True))
        b2_h_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(act_h2, act_h2_s), \
                0, True))
        
        # 세 번째 은닉층
        W3_add  = tf.multiply(lr_rbm/size_batch, tf.subtract(tf.matmul(tf.transpose(act_h2_s), \
                  act_h3), tf.matmul(tf.transpose(h2_s), act_h3_s)))  
        b3_i_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(act_h2_s, h2_s), \
                   0, True))  
        b3_h_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(act_h3, act_h3_s), \
                 0, True))  
        
        # 네 번째 은닉층
        W4_add  = tf.multiply(lr_rbm/size_batch, tf.subtract(tf.matmul(tf.transpose(act_h3_s), \
                  act_h4), tf.matmul(tf.transpose(h3_s), act_h4_s)))  
        b4_i_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(act_h3_s, h3_s), \
                   0, True))  
        b4_h_add = tf.multiply(lr_rbm/size_batch, tf.reduce_sum(tf.subtract(act_h4, act_h4_s), \
                 0, True))  
        
        updt = [W1.assign_add(W1_add), b1_i.assign_add(b1_i_add), b1_h.assign_add(b1_h_add),\
                W2.assign_add(W2_add), b2_i.assign_add(b2_i_add), b2_h.assign_add(b2_h_add),\
                W3.assign_add(W3_add), b3_i.assign_add(b3_i_add), b3_h.assign_add(b3_h_add),\
                W4.assign_add(W4_add), b4_i.assign_add(b4_i_add), b4_h.assign_add(b4_h_add)] 
        
        #-------------------------------------------------------------
        # 소프트맥스 층을 추가한 분류용-DBN 을 위한 연산과정
        #------------------------------------------------------------- 
        
        logits = tf.matmul(act_h4,W_c) + b_c
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))
        optimizer = tf.train.AdamOptimizer(learning_rate=lr_class).minimize(cost)
        correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        #-------------------------------------------------------------                   
        # RBM을 쌓아 올려가며 DBN을 학습하는 텐서플로 그래프 실행   
        #-------------------------------------------------------------
        with tf.Session() as sess:
            # Initialize the variables of the Model
            init = tf.global_variables_initializer()
            sess.run(init)
            
            n_batch = int(len(x_train)/batch_size)
            # Start the training 
            for epoch in range(n_epoch):
                # Loop over all batches
                for i in range(n_batch):
                    batch_xs = x_train[i*batch_size:(i+1)*batch_size]
                    # ++ 이미지 데이터를 784차원 벡터로 평탄화
                    batch_xs = batch_xs.reshape((-1, 784))
                    batch_xs = (batch_xs > 0)*1
                    # Run the weight update 
                    _ = sess.run(updt, feed_dict={x:batch_xs})
                    
                # Display the running step 
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1))
                          
            print("RBM training Completed !")
        #--------------------------------------------------------------
        # 소프트맥스 층을 추가한 분류용-DBN 학습 및 예측
        #--------------------------------------------------------------
            for i in range(n_iter):
                indices = np.random.randint(len(x_train), size=batch_size)
                batch_x = x_train[indices]
                batch_y = y_train[indices]
        
                # ++ 이미지 데이터를 784차원 벡터로 평탄화
                batch_x = batch_x.reshape((-1, 784))
                # ++ 레이블 데이터를 원-핫 인코딩
                batch_y = np.eye(10)[batch_y]
            
                # 최적화 과정 실행
                sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
                if i % 10 == 0:
                    # MINIST 훈련용 이미지의 배치에 대한 손실과 정확도를 계산
                    tr_loss, tr_acc = sess.run([cost, accuracy], 
                    	                    feed_dict={x: batch_x, y: batch_y})
                    print("Iter " + str(i) + ", Minibatch Loss= " + \
                          "{:.6f}".format(tr_loss) + ", Training Accuracy= " + \
                          "{:.5f}".format(tr_acc))
                
            print("Optimization Finished!")
        
            # MINIST 검정용 이미지에 대한 정확도 계산 
            print("Testing Accuracy:", \
                sess.run(accuracy, feed_dict={x: x_test.reshape((-1, 784)), #타입 수정
                                              y: np.eye(10)[y_test]})) #타입 수정
        
            sess.close()
             
        print("Hidden Layer 2 Nodes:", n_hidden2, "Hidden Layer 3 Nodes:", n_hidden3, "Accuracy:", accuracy)


Epoch: 0001
Epoch: 0002
Epoch: 0003
Epoch: 0004
Epoch: 0005
Epoch: 0006
Epoch: 0007
Epoch: 0008
Epoch: 0009
Epoch: 0010
Epoch: 0011
Epoch: 0012
Epoch: 0013
Epoch: 0014
Epoch: 0015
Epoch: 0016
Epoch: 0017
Epoch: 0018
Epoch: 0019
Epoch: 0020
Epoch: 0021
Epoch: 0022
Epoch: 0023
Epoch: 0024
Epoch: 0025
Epoch: 0026
Epoch: 0027
Epoch: 0028
Epoch: 0029
Epoch: 0030
Epoch: 0031
Epoch: 0032
Epoch: 0033
Epoch: 0034
Epoch: 0035
Epoch: 0036
Epoch: 0037
Epoch: 0038
Epoch: 0039
Epoch: 0040
Epoch: 0041
Epoch: 0042
Epoch: 0043
Epoch: 0044
Epoch: 0045
Epoch: 0046
Epoch: 0047
Epoch: 0048
Epoch: 0049
Epoch: 0050
Epoch: 0051
Epoch: 0052
Epoch: 0053
Epoch: 0054
Epoch: 0055
Epoch: 0056
Epoch: 0057
Epoch: 0058
Epoch: 0059
Epoch: 0060
Epoch: 0061
Epoch: 0062
Epoch: 0063
Epoch: 0064
Epoch: 0065
Epoch: 0066
Epoch: 0067
Epoch: 0068
Epoch: 0069
Epoch: 0070
Epoch: 0071
Epoch: 0072
Epoch: 0073
Epoch: 0074
Epoch: 0075
Epoch: 0076
Epoch: 0077
Epoch: 0078
Epoch: 0079
Epoch: 0080
Epoch: 0081
Epoch: 0082
Epoch: 0083
Epo